# 【双归一化网络】训练DINO

## 20230509log
* 基于NB01。预计加入tensorboard可视化
* 去除了测试部分的代码，使得整体更加简洁

## 20230513log 达到较为简洁的程度

* 【整理】把数据集、投射层、DINO数据增强放到了共享文件里面（TerrainDataset, Proj_layer, DataAugmentationDINO）
- 【理解】理清楚了训练（NB03）和测试（NB05）中使用的数据增强，
    - 训练使用DINOAug返回10张图像的list和标签组成的元组（list(tensor1, tensor2……), index）
    - 测试使用简单的transform返回一个tensor和标签组成的元组（tensor, index）
* 【删除】删除了Xcit的代码，因为并未使用它
* 【更改】TerrainDataset。**TerrainDataset中多做一步**，判断每次是增强了多个还是一个，多个的话返回list，一个直接返回tensor

## 20230525log 

* 【描述】【到NB13】改为双结构网络

## 20230528log

* 【描述】【NB13】加入EEnT支持
* 【使用】
    * 1 只需要改变 "--arch" 参数就可以改变训练时的架构了
    * 2 dataset_path变量：确保训练数据集是想要的
    * 3 "--num_ele_slice"参数为EEnT网络专有的，最好保持不变，在调试的时候再更改，确保后续步骤（NB14到16中），这一切片数量参数都保持一致才能运行

## 20230605log

* 【特性】加入连续运行Nb15编码存储与NB16匹配
    
## 20230714log

* 【特性】加入Py10中使用slice、per_li信息的兼容

## 20230718log

* 【特性】加入训练时给全局归一化数据增强的机制（TerrainDataset_BiNorm_Noise_on_g）

In [1]:
from Py01shared_code import TerrainDataset, Proj_layer, DataAugmentationDINO, get_args_parser, get_netpath_unique
from Py01shared_code import AddGaussianNoise, AddGaussianNoise_Pre_Snr, AddPepperNoise
from Py01shared_code import DataAugmentationDINO_ADJ, DataAugmentationDINO_ADJ_V2, DataAugmentationDINO_ADJ_V3
from Py06BiNet import MultiCropWrapper_Bi, DINOHead_Bi, TBiNet, TerrainDataset_BiNorm, TerrainDataset_BiNorm_Noise_on_g, get_dataset_minmax, TBiNet_Res
from Py07EEnT import ElevationTransformer, EEnT_tiny
from Py08EEnT_NO2 import get_dataset_per, ElevationTransformer_NO2, EEnT_NO2_tiny
import Py10Bi_model
from Py13_SA_FETM import get_bof_voc, get_dataset_var_per

In [2]:
import argparse
from torchvision import transforms as pth_transforms

In [3]:
# dataset_path = './Data_geo/20230517【补充多步长切分数据】/low-step-20'
dataset_path = "./Data_geo/深度学习地形数据/train"

args_self = [
"--lr", "0.0005",
'--dist_url', 'tcp://localhost:10001' ,
"--num_workers", "1",
'--batch_size_per_gpu', '45', # 64正好跑满24G显存(加上了双归一化数据及后不行了) # 对于SA_FETM_bi，50比较保险
'--arch', "GRET_ELE_DB_NO2_bi", # 一般可选 BiNet_NO1、EEnT_tiny_NO1、EEnT_tiny_NO2, BiNet_Res_NO1, 与Py10中的网络 如 TBiNet_Res_NO2_bi、SA_FETM_bi、EEnT_NO3_AT_bi
"--num_ele_slice", "20000", # 专为EEnT中的高程嵌入使用的参数 
"--epochs", "100"
]
parser = argparse.ArgumentParser('DINO', parents=[get_args_parser()])
args = parser.parse_args(args = args_self)

In [4]:
load_bof_voc = True

In [5]:
# transform_ADJ = DataAugmentationDINO_ADJ_V3( # 基于最初始版本的DINOAug更改
#     args.global_crops_scale,
#     args.local_crops_scale,
#     args.local_crops_number,
# #     USE_Noise_Aug=False, # 默认值为True，也即注释后，为True
# #     Use_HFlip_aug=False,
# #     Use_VFlip_aug=False,
# #     Use_Rotate_aug=False,
# #     Rotate_angle = 1,
# )

In [6]:
transform_ADJ = DataAugmentationDINO_ADJ( # 基于最初始版本的DINOAug更改
    args.global_crops_scale,
    args.local_crops_scale,
    args.local_crops_number,
#     USE_Noise_Aug=False, # 默认值为True，也即注释后，为True
#     Use_HFlip_aug=False,
#     Use_VFlip_aug=False,
)

transform_G = pth_transforms.Compose([
    AddPepperNoise(0.9999, p=1.0),
    pth_transforms.ToTensor(),
#         pth_transforms.Normalize(mean=[0.485], std=[0.229]),
])

/root/miniconda3/lib/python3.8/site-packages/torchvision/transforms/transforms.py:803: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [7]:
# transform_ADJ = DataAugmentationDINO( # 原始版本的增强，不修改
#     args.global_crops_scale,
#     args.local_crops_scale,
#     args.local_crops_number,
# )

In [8]:
def REA_gating_record(epoch, teacher, _):
#     print(args.arch)
    if args.arch[0:4] == "GRET":
        # 如果为0，记录头
        if epoch == 0 : 
            f = open("./【输出】NB13_gating_para.csv", "w")
            f.write("epochs, ")
            for i,block in enumerate(teacher.backbone.blocks):
                for name,para in block.named_parameters():
                    if name == "gating_param":
                        f.write(str(i+1) + ", ")
            f.write("\n")
            f.close()
        
        # 记录每一行（每一个epoch）
        f = open("./【输出】NB13_gating_para.csv", "a")
        f.write(str(epoch)+"," )
        for _, block in enumerate(teacher.backbone.blocks):
            for name,para in block.named_parameters():
            #     print(name)
                if name == "gating_param":
        #             print("gating_param = ", torch.sigmoid(para))
                    f.write(str(float(torch.sigmoid(para))) + ", ")
        f.write("\n")
        f.close()
    else:
        pass

In [9]:
import argparse
import os
import sys
import datetime
import time
import math
import json
from pathlib import Path

import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision import models as torchvision_models
from torch.utils.tensorboard import SummaryWriter   

import utils
import vision_transformer as vits
from functools import partial
from utils import trunc_normal_


# from functools import partial
# from torch.utils.data import _utils

# from timm.models.vision_transformer import _cfg, Mlp
# from timm.models.registry import register_model
# from timm.models.layers import DropPath, trunc_normal_, to_2tuple
from torchvision.transforms import ToPILImage

from einops import rearrange

# def trunc_normal_(tensor, mean=0., std=1., a=-2., b=2.):
#     # type: (Tensor, float, float, float, float) -> Tensor
#     return _no_grad_trunc_normal_(tensor, mean, std, a, b)

#定义投影头及其前向传播并初始化其参数
class DINOHead(nn.Module):
    def __init__(self, in_dim, out_dim, use_bn=False, norm_last_layer=True, nlayers=3, hidden_dim=2048, bottleneck_dim=256):
        super().__init__()
        nlayers = max(nlayers, 1)
        if nlayers == 1:
            self.mlp = nn.Linear(in_dim, bottleneck_dim)
        else:
            layers = [nn.Linear(in_dim, hidden_dim)]
            if use_bn:
                layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(nn.GELU())
            for _ in range(nlayers - 2):
                layers.append(nn.Linear(hidden_dim, hidden_dim))
                if use_bn:
                    layers.append(nn.BatchNorm1d(hidden_dim))
                layers.append(nn.GELU())
            layers.append(nn.Linear(hidden_dim, bottleneck_dim))
            self.mlp = nn.Sequential(*layers)
        self.apply(self._init_weights)
        self.last_layer = nn.utils.weight_norm(nn.Linear(bottleneck_dim, out_dim, bias=False))
        self.last_layer.weight_g.data.fill_(1)
        if norm_last_layer:
            self.last_layer.weight_g.requires_grad = False

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.mlp(x)
        x = nn.functional.normalize(x, dim=-1, p=2)
        x = self.last_layer(x)
        return x

class MultiCropWrapper(nn.Module):
    """
    Perform forward pass separately on each resolution input.
    The inputs corresponding to a single resolution are clubbed and single
    forward is run on the same resolution inputs. Hence we do several
    forward passes = number of different resolutions used. We then
    concatenate all the output features and run the head forward on these
    concatenated features.
    """
    def __init__(self, backbone, head):
        super(MultiCropWrapper, self).__init__()
        # disable layers dedicated to ImageNet labels classification
        backbone.fc, backbone.head = nn.Identity(), nn.Identity()
        self.backbone = backbone
        self.head = head

    def forward(self, x):
        # convert to list
        if not isinstance(x, list):
            x = [x]
        idx_crops = torch.cumsum(torch.unique_consecutive(
            torch.tensor([inp.shape[-1] for inp in x]),
            return_counts=True,
        )[1], 0)
        start_idx, output = 0, torch.empty(0).to(x[0].device)
        for end_idx in idx_crops:
            _out = self.backbone(torch.cat(x[start_idx: end_idx]))
            # The output is a tuple with XCiT model. See:
            # https://github.com/facebookresearch/xcit/blob/master/xcit.py#L404-L405
            if isinstance(_out, tuple):
                _out = _out[0]
            # accumulate outputs
            output = torch.cat((output, _out))
            start_idx = end_idx
        # Run the head forward on the concatenated features.
        return self.head(output)

def get_params_groups(model):
    regularized = []
    not_regularized = []
    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue
        # we do not regularize biases nor Norm parameters
        if name.endswith(".bias") or len(param.shape) == 1:
            not_regularized.append(param)
        else:
            regularized.append(param)
    return [{'params': regularized}, {'params': not_regularized, 'weight_decay': 0.}]
  
    
def bool_flag(s):
    """
    Parse boolean arguments from the command line.从命令行解析布尔参数。
    """
    FALSY_STRINGS = {"off", "false", "0"}
    TRUTHY_STRINGS = {"on", "true", "1"}
    if s.lower() in FALSY_STRINGS:
        return False
    elif s.lower() in TRUTHY_STRINGS:
        return True
    else:
        raise argparse.ArgumentTypeError("invalid value for a boolean flag")

torchvision_archs = sorted(name for name in torchvision_models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(torchvision_models.__dict__[name]))



In [10]:
class Proj_layer(nn.Module):
    def __init__(self):
        super(Proj_layer, self).__init__()
        self.patch_size = 224
    #         self.params=nn.Parameter(torch.randn(4, 1))
        self.conv_2d = nn.Conv2d(1, 192, kernel_size=(16, 16), stride=(16, 16))
    def forward(self, x):
        x = self.conv_2d(x)
        x = x.transpose(1,3)
        x = x.squeeze(2)
        x = rearrange(x, 'b c h w -> b (c h) w')
        return x

In [11]:
def train_dino(args):
    utils.init_distributed_mode(args)
    utils.fix_random_seeds(args.seed)
    print("git:\n  {}\n".format(utils.get_sha()))
    print("\n".join("%s: %s" % (k, str(v)) for k, v in sorted(dict(vars(args)).items())))
    cudnn.benchmark = True

    # ============ preparing data ... ============ 准备数据
#     transform_ADJ = DataAugmentationDINO(
#         args.global_crops_scale,
#         args.local_crops_scale,
#         args.local_crops_number,
#     )


    global_min, global_max = get_dataset_minmax(dataset_path)
    dataset_train = TerrainDataset_BiNorm_Noise_on_g(dataset_path, transform_ADJ, transform_G, global_min, global_max)
    print("两种归一化的数据集构建完成")
    # 获取bof特征
    if load_bof_voc:
        bof_voc, bof_stdSlr = torch.load("./bof_voc/voc.pth")
        voc_k = bof_voc.shape[0]
        print("voc_k = ", voc_k, " (代表sift_bof特征维度为此值，后续的嵌入会沿用这一切分维度)")
        print("成功加载bof_voc")
    else:
        voc_k = 100
        bof_voc, bof_stdSlr = get_bof_voc(dataset_path, using_num = 2000, voc_k = voc_k)
        torch.save((bof_voc, bof_stdSlr), "./bof_voc/voc.pth")
    # 获取高程分位数
    _, _, per_li = get_dataset_per(dataset_path, args.num_ele_slice)
    print("获取数据集高程分位数完成")
    
    _, _, per_li_var = get_dataset_var_per(dataset_path, args.num_ele_slice)
    print("获取数据集高程分位数(var)完成")
#     dataset_train = TerrainDataset(dataset_path, transform = transform)
#     dataset_test = TerrainDataset('./Data_geo/深度学习地形数据/test')

        

    train_sampler = torch.utils.data.distributed.DistributedSampler(dataset_train)
#     test_sampler = torch.utils.data.distributed.DistributedSampler(dataset_test)
    
#     dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=args.batch_size_per_gpu, sampler=train_sampler, num_workers=args.num_workers, pin_memory=True, drop_last=True)
    data_loader = torch.utils.data.DataLoader(dataset_train, batch_size=args.batch_size_per_gpu, sampler=train_sampler, num_workers=args.num_workers, pin_memory=True, drop_last=True)
#     dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=args.batch_size_per_gpu, sampler=test_sampler, num_workers=args.num_workers, pin_memory=True, drop_last=True)

#     print(f"Data loaded: there are {len(dataset_train),len(dataset_test)} images.")
    print(f"Data loaded: there are {len(dataset_train)} images.")

    # ============ building student and teacher networks ... ============构建网络
    # we changed the name DeiT-S for ViT-S to avoid confusions
    args.arch = args.arch.replace("deit", "vit")
    # if the network is a Vision Transformer (i.e. vit_tiny, vit_small, vit_base)
    if args.arch in vits.__dict__.keys():
        student = vits.__dict__[args.arch](
            patch_size=args.patch_size,
            drop_path_rate=args.drop_path_rate,  # stochastic depth
        )
        teacher = vits.__dict__[args.arch](patch_size=args.patch_size)
        embed_dim = student.embed_dim
            # 维度更改
        PatchEmbed_1ch = Proj_layer()
        teacher.patch_embed = PatchEmbed_1ch
        PatchEmbed_1ch2 = Proj_layer()
        student.patch_embed = PatchEmbed_1ch2
    # if the network is a XCiT
    elif args.arch in torch.hub.list("facebookresearch/xcit:main"):
        student = torch.hub.load('facebookresearch/xcit:main', args.arch,pretrained=False, drop_path_rate=args.drop_path_rate)
        teacher = torch.hub.load('facebookresearch/xcit:main', args.arch, pretrained=False)
        embed_dim = student.embed_dim
    # otherwise, we check if the architecture is in torchvision models
    elif args.arch in torchvision_models.__dict__.keys():
        student = torchvision_models.__dict__[args.arch]()
        teacher = torchvision_models.__dict__[args.arch]()
        embed_dim = student.fc.weight.shape[1]
    elif args.arch == "BiNet_NO1":
        student = TBiNet()
        teacher = TBiNet()
        embed_dim = student.embed_dim
    elif args.arch == "EEnT_tiny_NO1":
        student = EEnT_tiny(num_ele_slice = args.num_ele_slice, in_chans=1)
        teacher = EEnT_tiny(num_ele_slice = args.num_ele_slice, in_chans=1)
        embed_dim = student.embed_dim
    elif args.arch == "EEnT_tiny_NO2":
        student = EEnT_NO2_tiny(num_ele_slice = args.num_ele_slice, per_li = per_li, in_chans=1)
        teacher = EEnT_NO2_tiny(num_ele_slice = args.num_ele_slice, per_li = per_li, in_chans=1)
        embed_dim = student.embed_dim
    elif args.arch == "BiNet_Res_NO1":
        student = TBiNet_Res(conv_em_dim = 16)
        teacher = TBiNet_Res(conv_em_dim = 16)
        embed_dim = student.embed_dim
    elif args.arch in Py10Bi_model.__dict__.keys():
        try:
            student = Py10Bi_model.__dict__[args.arch]() # 调用函数
            teacher = Py10Bi_model.__dict__[args.arch]()
            embed_dim = student.embed_dim
        except: # 兼容需要slice和分位数列表的网络
            student = Py10Bi_model.__dict__[args.arch](num_ele_slice=args.num_ele_slice, per_li=per_li, per_li_var=per_li_var, 
                                                       bof_voc=bof_voc, bof_stdSlr=bof_stdSlr, in_chans=1, voc_k = voc_k,
                                                      g_min = global_min, g_max = global_max) # 调用函数
            teacher = Py10Bi_model.__dict__[args.arch](num_ele_slice=args.num_ele_slice, per_li=per_li, per_li_var=per_li_var,
                                                       bof_voc=bof_voc, bof_stdSlr=bof_stdSlr, in_chans=1, voc_k = voc_k,
                                                      g_min = global_min, g_max = global_max)
            embed_dim = student.embed_dim
    else:
        print(f"Unknow architecture: {args.arch}")
        assert 0


#     print(teacher)
    # multi-crop wrapper handles forward with inputs of different resolutions多裁剪器
    student = MultiCropWrapper_Bi(student, DINOHead_Bi(
        embed_dim,
        args.out_dim,
        use_bn=args.use_bn_in_head,
        norm_last_layer=args.norm_last_layer,
    ))
    teacher = MultiCropWrapper_Bi(
        teacher,
        DINOHead(embed_dim, args.out_dim, args.use_bn_in_head),
    )
    
    # move networks to gpu
    student, teacher = student.cuda(), teacher.cuda()
    
    # synchronize batch norms (if any)同步批处理规范
    if utils.has_batchnorms(student):
        student = nn.SyncBatchNorm.convert_sync_batchnorm(student)
        teacher = nn.SyncBatchNorm.convert_sync_batchnorm(teacher)

        # we need DDP wrapper to have synchro batch norms working...我们需要DDP包装有同步批处理规范工作
        teacher = nn.parallel.DistributedDataParallel(teacher, device_ids=[args.gpu])
        teacher_without_ddp = teacher.module
    else:
        # teacher_without_ddp and teacher are the same thing
        teacher_without_ddp = teacher
    student = nn.parallel.DistributedDataParallel(student, device_ids=[args.gpu])
    
    # teacher and student start with the same weights
    teacher_without_ddp.load_state_dict(student.module.state_dict())
    
    # there is no backpropagation through the teacher, so no need for gradients教师网络无需更新梯度
    for p in teacher.parameters():
        p.requires_grad = False
    print(f"Student and Teacher are built: they are both {args.arch} network.")

    # ============ preparing loss ... ============损失函数
    dino_loss = DINOLoss(
        args.out_dim,
        args.local_crops_number + 2,  # total number of crops = 2 global crops + local_crops_number
        args.warmup_teacher_temp,
        args.teacher_temp,
        args.warmup_teacher_temp_epochs,
        args.epochs,
    ).cuda()
    
    # ============ preparing optimizer ... ============优化器
    params_groups = utils.get_params_groups(student)
    if args.optimizer == "adamw":
        optimizer = torch.optim.AdamW(params_groups)  # to use with ViTs
    elif args.optimizer == "sgd":
        optimizer = torch.optim.SGD(params_groups, lr=0, momentum=0.9)  # lr is set by scheduler
    elif args.optimizer == "lars":
        optimizer = utils.LARS(params_groups)  # to use with convnet and large batches
    # for mixed precision training
    fp16_scaler = None
    if args.use_fp16:
        fp16_scaler = torch.cuda.amp.GradScaler()

    
    # ============ init schedulers ... ============初始化调度器
    lr_schedule = utils.cosine_scheduler(
        args.lr * (args.batch_size_per_gpu * utils.get_world_size()) / 256.,  # linear scaling rule线性化比例缩小
        args.min_lr,
        args.epochs, len(data_loader),
        warmup_epochs=args.warmup_epochs,
    )
    wd_schedule = utils.cosine_scheduler(
        args.weight_decay,
        args.weight_decay_end,
        args.epochs, len(data_loader),
    )
    # momentum parameter is increased to 1. during training with a cosine schedule当训练使用余弦调度时，动量参数增加到1
    momentum_schedule = utils.cosine_scheduler(args.momentum_teacher, 1,
                                               args.epochs, len(data_loader))
    print(f"Loss, optimizer and schedulers ready.")

    # ============ optionally resume training ... ============可选择的恢复训练
    to_restore = {"epoch": 0}
    utils.restart_from_checkpoint(
        os.path.join(args.output_dir, "checkpoint.pth"),
        run_variables=to_restore,
        student=student,
        teacher=teacher,
        optimizer=optimizer,
        fp16_scaler=fp16_scaler,
        dino_loss=dino_loss,
    )
    start_epoch = to_restore["epoch"]

    start_time = time.time()
    print("Starting DINO training !")
    for epoch in range(start_epoch, args.epochs):
        data_loader.sampler.set_epoch(epoch)

        # ============ training one epoch of DINO ... ============训练一个周期
        train_stats = train_one_epoch(student, teacher, teacher_without_ddp, dino_loss,
            data_loader, optimizer, lr_schedule, wd_schedule, momentum_schedule,
            epoch, fp16_scaler, args)

        # ============ writing logs ... ============显示、存储状态
        save_dict = {
            'student': student.state_dict(),
            'teacher': teacher.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch + 1,
            'args': args,
            'dino_loss': dino_loss.state_dict(),
        }
        if fp16_scaler is not None:
            save_dict['fp16_scaler'] = fp16_scaler.state_dict()
        utils.save_on_master(save_dict, os.path.join(args.output_dir, 'checkpoint.pth'))
        if args.saveckp_freq and epoch % args.saveckp_freq == 0:
            utils.save_on_master(save_dict, os.path.join(args.output_dir, f'checkpoint{epoch:04}.pth'))
        log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                     'epoch': epoch}
        if utils.is_main_process():
            with (Path(args.output_dir) / "log.txt").open("a") as f:
                f.write(json.dumps(log_stats) + "\n")
    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Training time {}'.format(total_time_str))


In [12]:
def train_one_epoch(student, teacher, teacher_without_ddp, dino_loss, data_loader,
                    optimizer, lr_schedule, wd_schedule, momentum_schedule,epoch,
                    fp16_scaler, args):
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = 'Epoch: [{}/{}]'.format(epoch, args.epochs)
    log_every_num = (len(data_loader) * 100) // 800
    # 每个epoch存储REA模块中的Gating Para
    try:
        REA_gating_record(epoch, teacher, student)
    except:
        pass
    
    for it, (images, im_global_norm, _) in enumerate(metric_logger.log_every(data_loader, 10, header)):
        # update weight decay and learning rate according to their schedule根据他们的时间表更新权重衰减和学习率
        it = len(data_loader) * epoch + it  # global training iteration全局训练迭代
        for i, param_group in enumerate(optimizer.param_groups):
            param_group["lr"] = lr_schedule[it]
            if i == 0:  # 只有第一组是正则化的
                param_group["weight_decay"] = wd_schedule[it]
#         print(type(images))
        # move images to gpu
        images = [im.cuda(non_blocking=True) for im in images]
        im_global_norm = im_global_norm.cuda()
        # teacher and student forward passes + compute dino loss
        with torch.cuda.amp.autocast(fp16_scaler is not None):
            teacher_output = teacher(images[:2], im_global_norm)  # only the 2 global views pass through the teacher
            student_output = student(images, im_global_norm)
            loss = dino_loss(student_output, teacher_output, epoch)

        if not math.isfinite(loss.item()):
            print("Loss is {}, stopping training".format(loss.item()), force=True)
            sys.exit(1)

        # student update
        optimizer.zero_grad()
        param_norms = None
        if fp16_scaler is None:
            loss.backward()
            if args.clip_grad:
                param_norms = utils.clip_gradients(student, args.clip_grad)
            utils.cancel_gradients_last_layer(epoch, student,
                                              args.freeze_last_layer)
            optimizer.step()
        else:
            fp16_scaler.scale(loss).backward()
            if args.clip_grad:
                fp16_scaler.unscale_(optimizer)  # unscale the gradients of optimizer's assigned params in-place
                param_norms = utils.clip_gradients(student, args.clip_grad)
            utils.cancel_gradients_last_layer(epoch, student,
                                              args.freeze_last_layer)
            fp16_scaler.step(optimizer)
            fp16_scaler.update()
        

        
        # EMA update for the teacher使用EMA更新teacher
        with torch.no_grad():
            m = momentum_schedule[it]  # momentum parameter
            for param_q, param_k in zip(student.module.parameters(), teacher_without_ddp.parameters()):
                param_k.data.mul_(m).add_((1 - m) * param_q.detach().data)
        
#         for name, param in student.named_parameters(): # 调试没有梯度的层用
#             if param.grad is None:
#                 print(name)
        
        # logging日志
        torch.cuda.synchronize()
        metric_logger.update(loss=loss.item())
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])
        metric_logger.update(wd=optimizer.param_groups[0]["weight_decay"])
        if it % log_every_num == 0: # 总记录少于1000个点
            writer.add_scalar("loss", loss.item(), it)
            writer.add_scalar("epoch", epoch, it)
            writer.add_scalar("lr", optimizer.param_groups[0]["lr"], it)
            writer.add_scalar("wd", optimizer.param_groups[0]["weight_decay"], it)
    
    # gather the stats from all processes收集所有进程的统计信息
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    return {k: meter.global_avg for k, meter in metric_logger.meters.items()}


In [13]:
class DINOLoss(nn.Module):
    def __init__(self, out_dim, ncrops, warmup_teacher_temp, teacher_temp,
                 warmup_teacher_temp_epochs, nepochs, student_temp=0.1,
                 center_momentum=0.9):
        super().__init__()
        self.student_temp = student_temp
        self.center_momentum = center_momentum
        self.ncrops = ncrops
        self.register_buffer("center", torch.zeros(1, out_dim))
        # 我们对教师温度进行了预热，因为过高的温度会使培训在开始时不稳定
        self.teacher_temp_schedule = np.concatenate((
            np.linspace(warmup_teacher_temp,
                        teacher_temp, warmup_teacher_temp_epochs),
            np.ones(nepochs - warmup_teacher_temp_epochs) * teacher_temp
        ))

    def forward(self, student_output, teacher_output, epoch):
        """
        Cross-entropy between softmax outputs of the teacher and student networks.
        """
        student_out = student_output / self.student_temp
        student_out = student_out.chunk(self.ncrops)

        # teacher centering and sharpening
        temp = self.teacher_temp_schedule[epoch]
        teacher_out = F.softmax((teacher_output - self.center) / temp, dim=-1)
        teacher_out = teacher_out.detach().chunk(2)

        total_loss = 0
        n_loss_terms = 0
        for iq, q in enumerate(teacher_out):
            for v in range(len(student_out)):
                if v == iq:
                    # we skip cases where student and teacher operate on the same view我们跳过了学生和老师在同一个视图上操作的情况
                    continue
                loss = torch.sum(-q * F.log_softmax(student_out[v], dim=-1), dim=-1)
                total_loss += loss.mean()
                n_loss_terms += 1
        total_loss /= n_loss_terms
        self.update_center(teacher_output)
        return total_loss

    @torch.no_grad()
    def update_center(self, teacher_output):
        """
        Update center used for teacher output.
        """
        batch_center = torch.sum(teacher_output, dim=0, keepdim=True)
        dist.all_reduce(batch_center)
        batch_center = batch_center / (len(teacher_output) * dist.get_world_size())

        # ema update
        self.center = self.center * self.center_momentum + batch_center * (1 - self.center_momentum)


In [14]:
writer = SummaryWriter('/root/tf-logs') # init write obj 初始化tensorboard的记录


In [ ]:
# %tb ==========主函数===============
if __name__ == '__main__':
#     parser = argparse.ArgumentParser('DINO', parents=[get_args_parser()])
#     args = parser.parse_args(args = args_self)
    Path(args.output_dir).mkdir(parents=True, exist_ok=True)
    train_dino(args)

Will run the code on one GPU.
| distributed init (rank 0): tcp://localhost:10001


fatal: not a git repository (or any parent up to mount point /root)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


git:
  sha: N/A, status: clean, branch: N/A

arch: GRET_ELE_DB_NO2_bi
batch_size_per_gpu: 45
clip_grad: 3.0
data_path: /path/to/imagenet/train/
dist_url: tcp://localhost:10001
drop_path_rate: 0.1
epochs: 100
freeze_last_layer: 1
global_crops_scale: (0.4, 1.0)
gpu: 0
local_crops_number: 8
local_crops_scale: (0.05, 0.4)
local_rank: 0
lr: 0.0005
min_lr: 1e-06
momentum_teacher: 0.996
norm_last_layer: True
num_ele_slice: 20000
num_workers: 1
optimizer: adamw
out_dim: 65536
output_dir: .
patch_size: 16
rank: 0
saveckp_freq: 20
seed: 0
teacher_temp: 0.04
use_bn_in_head: False
use_fp16: True
warmup_epochs: 10
warmup_teacher_temp: 0.04
warmup_teacher_temp_epochs: 0
weight_decay: 0.04
weight_decay_end: 0.4
world_size: 1
获取数据集最大最小值……
数据集最大最小值为(-9655.287, 5201.252)
两种归一化的数据集构建完成
voc_k =  100  (代表sift_bof特征维度为此值，后续的嵌入会沿用这一切分维度)
成功加载bof_voc
获取数据集分位数……
数据集最大最小值为(-9655.287, 5201.252)
获取数据集高程分位数完成
获取数据集分位数……(VAR)
数据集最大最小值为(-9655.287, 5201.252)
获取数据集高程分位数(var)完成
Data loaded: there are 8363 images.


Using cache found in /root/.cache/torch/hub/facebookresearch_xcit_main


Student and Teacher are built: they are both GRET_ELE_DB_NO2_bi network.
Loss, optimizer and schedulers ready.
Found checkpoint at ./checkpoint.pth
=> loaded 'student' from checkpoint './checkpoint.pth' with msg <All keys matched successfully>
=> loaded 'teacher' from checkpoint './checkpoint.pth' with msg <All keys matched successfully>
=> loaded 'optimizer' from checkpoint: './checkpoint.pth'
=> loaded 'fp16_scaler' from checkpoint: './checkpoint.pth'
=> loaded 'dino_loss' from checkpoint './checkpoint.pth' with msg <All keys matched successfully>
Starting DINO training !


/root/miniconda3/lib/python3.8/site-packages/torch/nn/functional.py:3454: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


Epoch: [2/100]  [  0/185]  eta: 0:22:46  loss: 10.515423 (10.515423)  lr: 0.000018 (0.000018)  wd: 0.040355 (0.040355)  time: 7.384190  data: 4.497261  max mem: 19093
Epoch: [2/100]  [ 10/185]  eta: 0:08:55  loss: 10.431251 (10.435861)  lr: 0.000018 (0.000018)  wd: 0.040365 (0.040365)  time: 3.061275  data: 1.626753  max mem: 19201
Epoch: [2/100]  [ 20/185]  eta: 0:07:49  loss: 10.426844 (10.427522)  lr: 0.000018 (0.000018)  wd: 0.040375 (0.040375)  time: 2.621343  data: 1.347811  max mem: 19201
Epoch: [2/100]  [ 30/185]  eta: 0:07:17  loss: 10.397007 (10.412457)  lr: 0.000019 (0.000018)  wd: 0.040395 (0.040385)  time: 2.691120  data: 1.430761  max mem: 19201
Epoch: [2/100]  [ 40/185]  eta: 0:06:44  loss: 10.374393 (10.402686)  lr: 0.000019 (0.000019)  wd: 0.040415 (0.040395)  time: 2.730824  data: 1.420173  max mem: 19201
Epoch: [2/100]  [ 50/185]  eta: 0:06:14  loss: 10.349962 (10.390920)  lr: 0.000019 (0.000019)  wd: 0.040436 (0.040405)  time: 2.690662  data: 1.378794  max mem: 1920

# 训练后处理

In [ ]:
# 移动网络到存到文件夹，并重命名（自动累加）
import shutil
save_net_name_unique, netname = get_netpath_unique("【" + args.arch  + "" +"】")
shutil.move("./checkpoint.pth", save_net_name_unique) # 将其移动到应该的位置

# 继续【NB14】BiNet_eval_based_on_knn.ipynb

In [ ]:
# # save_net_name_unique
# save_net_name_unique = './【checkpoint存档】/20230725【GRET_NO1_bi】DINO_3_01.pth'
# # save_net_name_unique = "./checkpoint.pth"

In [ ]:
pre_model_path_outside = save_net_name_unique
# net_name_outside = netname
arch_outside = args.arch
num_ele_slice_outside = args.num_ele_slice

In [ ]:
# print(dir())
# # 清除
# for var in list(globals().keys()):
#     if not var.startswith("_") and not var.endswith("_outside"):
#         del globals()[var]

In [ ]:
%run ./【NB14】BiNet_eval_based_on_knn.ipynb

# 继续【NB15】BiNet_Encode_map2Vec.ipynb

In [ ]:
# print(dir())
# # 清除
# for var in list(globals().keys()):
#     if not var.startswith("_") and not var.endswith("_outside"):
#         del globals()[var]

In [ ]:
import torch
torch.cuda.empty_cache() #使用memory_allocated前先清空一下cache
torch.cuda.memory_allocated()
torch.cuda.reset_max_memory_allocated()
torch.cuda.reset_peak_memory_stats()

In [ ]:
%run ./【NB15】BiNet_Encode_map2Vec.ipynb

# 继续【NB16】BiNet_Matching_Iter.ipynb

In [ ]:
# print(dir())
# # 清除
# for var in list(globals().keys()):
#     if not var.startswith("_") and not var.endswith("_outside"):
#         del globals()[var]

In [ ]:
import torch
torch.cuda.empty_cache() #使用memory_allocated前先清空一下cache
torch.cuda.memory_allocated()
torch.cuda.reset_max_memory_allocated()
torch.cuda.reset_peak_memory_stats()
# print(torch.cuda.memory_summary())

In [ ]:
%run ./【NB16】BiNet_Matching_Iter.ipynb